In [1]:
import shutil

def other_paths(scene_id,base_path,base_depth_path,mode="train") :
    if not os.path.exists(os.path.join(base_depth_path, scene_id, 'dense0')):
        os.mkdir(os.path.join(base_depth_path, scene_id, 'dense0'))
    else :
        shutil.rmtree(os.path.join(base_depth_path, scene_id, 'dense0'))
        os.mkdir(os.path.join(base_depth_path, scene_id, 'dense0'))
    if not os.path.exists(os.path.join(base_depth_path, scene_id, 'dense0','depths')):
        os.mkdir(os.path.join(base_depth_path, scene_id, 'dense0','depths'))
    if not os.path.exists(os.path.join(base_depth_path, scene_id, 'dense0','calib')):
        os.mkdir(os.path.join(base_depth_path, scene_id, 'dense0','calib'))
    base_undistorted_sfm_path = os.path.join(
        base_path, 'data',mode
    )
    # print("base_undistorted_sfm_path - ",base_undistorted_sfm_path)
    undistorted_sparse_path = os.path.join(
        base_undistorted_sfm_path, scene_id, 'sfm'
    )
    # print("undistorted_sparse_path - ",undistorted_sparse_path)
    depths_path = os.path.join(
        base_depth_path, scene_id, 'dense0', 'depths'
    )
    
    images_path = os.path.join(
        base_undistorted_sfm_path, scene_id, 'images'
    )
    
    return base_undistorted_sfm_path,undistorted_sparse_path,depths_path,images_path

In [2]:
def other_changes() :
    if os.path.exists("/home/hemant/Downloads/amit/ALIKE_code/data/train/multi-temporal-temple-baalshamin/images/2015_10_15_19.png"):
        os.rename("/home/hemant/Downloads/amit/ALIKE_code/data/train/multi-temporal-temple-baalshamin/images/2015_10_15_19.png","/home/hemant/Downloads/amit/ALIKE_code/data/train/multi-temporal-temple-baalshamin/images/2015_10_15_19.12.50.png")

In [3]:
# Process images.txt
def check_images_exist(undistorted_sparse_path,images_path) :
    with open(os.path.join(undistorted_sparse_path, 'images.txt'), 'r') as f:
        raw = f.readlines()[4 :]  # skip the header
    
    image_names = []

    for idx, (image, points) in enumerate(zip(raw[:: 2], raw[1 :: 2])):
        image = image.split(' ')
    
        image_name = image[-1].strip('\n').lower().replace("-","_").replace("(","").replace(")","").replace("'","_").replace("__","_")
        if os.path.exists(os.path.join(images_path, image_name)):
            image_names.append(image_name)
        else :
            if image_name.endswith(".png") :
                find_image_name = image_name.replace(".png",".jpg")
            elif image_name.endswith(".jpg") :
                find_image_name = image_name.replace(".jpg",".png")

            if not os.path.exists(os.path.join(images_path, find_image_name)):
                print("{} not found . Converting from {} . This also not found".format(image_name,find_image_name))

            else :
                #open image in png format 
                img_png = Image.open(os.path.join(images_path, find_image_name))
                if img_png.mode in ("RGBA", "P"): 
                    img_png = img_png.convert("RGB")
                  
                #The image object is used to save the image in jpg format 
                img_png.save(os.path.join(images_path, image_name))
                os.remove(os.path.join(images_path, find_image_name))
                image_names.append(image_name)

In [4]:
import h5py
from PIL import Image
import torch
from LightGlue.lightglue import ALIKED
import kornia as K

def load_torch_image(fname, device=torch.device('cpu')):
    img = K.io.load_image(fname, K.io.ImageLoadType.RGB32, device=device)[None, ...]
    return img

def depth_file_create(images_path,depths_path) :

    image_list = [image_path for image_path in os.listdir(images_path) if ( image_path.endswith(".png") or image_path.endswith(".jpg") )]

    num_features = 4096
    resize_to = 1024
    device=torch.device('cuda')
    dtype = torch.float32 # ALIKED has issues with float16
    extractor = ALIKED(max_num_keypoints=num_features, detection_threshold=0.01, resize=resize_to).eval().to(device, dtype)
    
    for image in image_list :
        if image.endswith(".png") :
            save_path = os.path.join(depths_path, image.replace(".png",".h5"))
        else :
            save_path = os.path.join(depths_path, image.replace(".jpg",".h5"))
        img_path = os.path.join(images_path, image)
        
        # print('image size: %d bytes'%os.path.getsize(img_path))
        image0 = load_torch_image(img_path, device=device).to(dtype)
        feats0 = extractor.extract_dense_map(image0)  # auto-resize the image, disable with resize=None
        feats0[1][0,0,:,:]
        hf = h5py.File(save_path, 'a') # open a hdf5 file
        # img_np = np.array(Image.open(img_path))
        dense_data = feats0[1][0,0,:,:].detach().cpu()
        
        dset = hf.create_dataset('depth', data=dense_data)  # write the data to hdf5 file
        hf.close()  # close the hdf5 file
        del feats0,dense_data
        gc.collect()
        torch.cuda.empty_cache()
        # print('hdf5 file size: %d bytes'%os.path.getsize(save_path))

In [5]:
# Process cameras.txt
def process_cameras(undistorted_sparse_path) :
    with open(os.path.join(undistorted_sparse_path, 'cameras.txt'), 'r') as f:
        raw = f.readlines()[3 :]  # skip the header
    
    camera_intrinsics = {}
    for camera in raw:
        camera = camera.split(' ')
        camera_intrinsics[int(camera[0])] = [float(elem) for elem in camera[2 :]]
    
    return camera_intrinsics

In [6]:
# Process points3D.txt
def process_points3D(undistorted_sparse_path) :
    with open(os.path.join(undistorted_sparse_path, 'points3D.txt'), 'r') as f:
        raw = f.readlines()[3 :]  # skip the header
    
    points3D = {}
    for point3D in raw:
        point3D = point3D.split(' ')
        points3D[int(point3D[0])] = np.array([
            float(point3D[1]), float(point3D[2]), float(point3D[3])
        ])
    
    return points3D

In [7]:
# Process images.txt
def process_images(undistorted_sparse_path,images_path) :
    with open(os.path.join(undistorted_sparse_path, 'images.txt'), 'r') as f:
        raw = f.readlines()[4 :]  # skip the header
    
    image_id_to_idx = {}
    image_names = []
    raw_pose = []
    camera = []
    points3D_id_to_2D = []
    n_points3D = []
    for idx, (image, points) in enumerate(zip(raw[:: 2], raw[1 :: 2])):
        image = image.split(' ')
        points = points.split(' ')
    
        image_id_to_idx[int(image[0])] = idx
    
        image_name = image[-1].strip('\n').lower().replace("-","_").replace("(","").replace(")","").replace("'","_").replace("__","_")

        if os.path.exists(os.path.join(images_path, image_name)) :
            image_names.append(image_name)
        
            raw_pose.append([float(elem) for elem in image[1 : -2]])
            camera.append(int(image[-2]))
            current_points3D_id_to_2D = {}
            for x, y, point3D_id in zip(points[:: 3], points[1 :: 3], points[2 :: 3]):
                if int(point3D_id) == -1:
                    continue
                current_points3D_id_to_2D[int(point3D_id)] = [float(x), float(y)]
            points3D_id_to_2D.append(current_points3D_id_to_2D)
            n_points3D.append(len(current_points3D_id_to_2D))
    return image_id_to_idx,image_names,raw_pose,camera,points3D_id_to_2D,n_points3D

In [8]:
# Image and depthmaps paths
def image_depthmaps_path(image_names,images_path,depths_path):
    image_paths = []
    depth_paths = []
    for image_name in image_names:
        image_path = os.path.join(images_path, image_name)
       
        # Path to the depth file
        depth_path = os.path.join(
            depths_path, '%s.h5' % os.path.splitext(image_name)[0]
        )
        
        if os.path.exists(depth_path):
            # Check if depth map or background / foreground mask
            file_size = os.stat(depth_path).st_size
            # Rough estimate - 75KB might work as well
            if file_size < 100 * 1024:
                depth_paths.append(None)
                image_paths.append(None)
            else:
                depth_paths.append(depth_path[len(base_path) + 1 :])
                image_paths.append(image_path[len(base_path) + 1 :])
        else:
            depth_paths.append(None)
            image_paths.append(None)
    
    return image_paths,depth_paths

In [9]:
# Camera configuration
def camera_configuration(image_names,image_paths,camera_intrinsics,camera,raw_pose,points3D_id_to_2D,points3D):
    intrinsics = []
    poses = []
    principal_axis = []
    points3D_id_to_ndepth = []
    for idx, image_name in enumerate(image_names):
        if image_paths[idx] is None:
            intrinsics.append(None)
            poses.append(None)
            principal_axis.append([0, 0, 0])
            points3D_id_to_ndepth.append({})
            continue
    
        image_intrinsics = camera_intrinsics[camera[idx]]
        K = np.zeros([3, 3])
        K[0, 0] = image_intrinsics[0]
        K[0, 2] = image_intrinsics[3]
        K[1, 1] = image_intrinsics[1]
        K[1, 2] = image_intrinsics[4]
        K[2, 2] = 1
        intrinsics.append(K)
    
        image_pose = raw_pose[idx]
        qvec = image_pose[: 4]
        qvec = qvec / np.linalg.norm(qvec)
        w, x, y, z = qvec
        R = np.array([
            [
                1 - 2 * y * y - 2 * z * z,
                2 * x * y - 2 * z * w,
                2 * x * z + 2 * y * w
            ],
            [
                2 * x * y + 2 * z * w,
                1 - 2 * x * x - 2 * z * z,
                2 * y * z - 2 * x * w
            ],
            [
                2 * x * z - 2 * y * w,
                2 * y * z + 2 * x * w,
                1 - 2 * x * x - 2 * y * y
            ]
        ])
        principal_axis.append(R[2, :])
        t = image_pose[4 : 7]
        # World-to-Camera pose
        current_pose = np.zeros([4, 4])
        current_pose[: 3, : 3] = R
        current_pose[: 3, 3] = t
        current_pose[3, 3] = 1
        # Camera-to-World pose
        # pose = np.zeros([4, 4])
        # pose[: 3, : 3] = np.transpose(R)
        # pose[: 3, 3] = -np.matmul(np.transpose(R), t)
        # pose[3, 3] = 1
        poses.append(current_pose)
        if image_paths[idx].endswith(".jpg") :
            save_path = image_paths[idx].replace("images","dense0/calib").replace(".jpg",".h5")
        elif image_paths[idx].endswith(".png") :
            save_path = image_paths[idx].replace("images","dense0/calib").replace(".png",".h5")
        if os.path.exists(save_path) :
            os.remove(save_path)
        hf = h5py.File(save_path, 'a') # open a hdf5 file
        # img_np = np.array(Image.open(img_path))
        data={}
        data["K"]=K
        data["R"]=R
        data["T"]=t
        
        dset = hf.create_dataset('K', data=data["K"])  # write the data to hdf5 file
        dset = hf.create_dataset('R', data=data["R"])  # write the data to hdf5 file
        dset = hf.create_dataset('T', data=data["T"])  # write the data to hdf5 file
        hf.close()
        
        current_points3D_id_to_ndepth = {}
        for point3D_id in points3D_id_to_2D[idx].keys():
            p3d = points3D[point3D_id]
            current_points3D_id_to_ndepth[point3D_id] = (np.dot(R[2, :], p3d) + t[2]) / (.5 * (K[0, 0] + K[1, 1])) 
        points3D_id_to_ndepth.append(current_points3D_id_to_ndepth)
    principal_axis = np.array(principal_axis)
    angles = np.rad2deg(np.arccos(
        np.clip(
            np.dot(principal_axis, np.transpose(principal_axis)),
            -1, 1
        )
    ))

    return intrinsics,poses,principal_axis,points3D_id_to_ndepth

In [10]:
# Compute overlap score
def compute_overlap_score(n_images,image_paths,depth_paths,points3D_id_to_2D,points3D_id_to_ndepth):
    overlap_matrix = np.full([n_images, n_images], -1.)
    scale_ratio_matrix = np.full([n_images, n_images], -1.)
    for idx1 in range(n_images):
        if image_paths[idx1] is None or depth_paths[idx1] is None:
            continue
        for idx2 in range(idx1 + 1, n_images):
            if image_paths[idx2] is None or depth_paths[idx2] is None:
                continue
            matches = (
                points3D_id_to_2D[idx1].keys() &
                points3D_id_to_2D[idx2].keys()
            )
            min_num_points3D = min(
                len(points3D_id_to_2D[idx1]), len(points3D_id_to_2D[idx2])
            )
            overlap_matrix[idx1, idx2] = len(matches) / len(points3D_id_to_2D[idx1])  # min_num_points3D
            overlap_matrix[idx2, idx1] = len(matches) / len(points3D_id_to_2D[idx2])  # min_num_points3D
            if len(matches) == 0:
                continue
            points3D_id_to_ndepth1 = points3D_id_to_ndepth[idx1]
            points3D_id_to_ndepth2 = points3D_id_to_ndepth[idx2]
            nd1 = np.array([points3D_id_to_ndepth1[match] for match in matches])
            nd2 = np.array([points3D_id_to_ndepth2[match] for match in matches])
            min_scale_ratio = np.min(np.maximum(nd1 / nd2, nd2 / nd1))
            scale_ratio_matrix[idx1, idx2] = min_scale_ratio
            scale_ratio_matrix[idx2, idx1] = min_scale_ratio

    return overlap_matrix,scale_ratio_matrix

In [11]:
import torch
import gc

def creating_dataset(base_path,mode) :
    
    base_depth_path = os.path.join(
        base_path, 'data',mode
    )
    print("base_depth_path - ",base_depth_path)
    print("-"*117)
    scene_list = [directory for directory in os.listdir(base_depth_path) if os.path.isdir(os.path.join(base_depth_path,directory))]
    
    train_json_data = {}
    val_json_data = {}
    
    for scene in scene_list :
        print("scene_id - ",scene)
        print(f"GPU total used memory is {round(get_used_gpu_memory()[0]/1024,2)} GB")
        scene_id = scene

        base_undistorted_sfm_path,undistorted_sparse_path,depths_path,images_path = other_paths(scene_id,base_path,base_depth_path,mode)
        check_images_exist(undistorted_sparse_path,images_path)
        depth_file_create(images_path,depths_path)
        camera_intrinsics = process_cameras(undistorted_sparse_path)
        points3D = process_points3D(undistorted_sparse_path)
        image_id_to_idx,image_names,raw_pose,camera,points3D_id_to_2D,n_points3D = process_images(undistorted_sparse_path,images_path)
        image_paths,depth_paths = image_depthmaps_path(image_names,images_path,depths_path)
        intrinsics,poses,principal_axis,points3D_id_to_ndepth = camera_configuration(image_names,image_paths,camera_intrinsics,camera,raw_pose,points3D_id_to_2D,points3D)
        overlap_matrix,scale_ratio_matrix = compute_overlap_score(len(image_names),image_paths,depth_paths,points3D_id_to_2D,points3D_id_to_ndepth)

        if scene_id == "pond" :
            train_image_names = image_names[:1001]
            val_image_names = image_names[1001:]

            val_json_data[scene_id]={
                'image_path':images_path,
                'images':val_image_names,#[image.split("/")[-1].replace("jpg","png") for image in image_paths],
                'depth_path':depths_path,
                'calib_path':depths_path.replace("depths","calib"),
                'tuples':list(range(len(val_image_names))),#[int(image.split(".")[0].split("/")[-1]) for image in image_paths],#list(range(len(image_paths))),
            }
        else:
            train_image_names = image_names
        
        train_json_data[scene_id]={
            'image_path':images_path,
            'images':train_image_names,#[image.split("/")[-1].replace("jpg","png") for image in image_paths],
            'depth_path':depths_path,
            'calib_path':depths_path.replace("depths","calib"),
            'tuples':list(range(len(train_image_names))),#[int(image.split(".")[0].split("/")[-1]) for image in image_paths],#list(range(len(image_paths))),
        }

        gc.collect()
        torch.cuda.empty_cache()
        print("-"*117)
    return train_json_data,val_json_data

In [12]:
import json
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [13]:
import subprocess as sp
import os

def get_total_gpu_memory():
    command = "nvidia-smi --query-gpu=memory.total --format=csv"
    memory_total_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
    memory_total_values = [int(x.split()[0]) for i, x in enumerate(memory_total_info)]
    return memory_total_values

def get_free_gpu_memory():
    command = "nvidia-smi --query-gpu=memory.free --format=csv"
    memory_free_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
    memory_free_values = [int(x.split()[0]) for i, x in enumerate(memory_free_info)]
    return memory_free_values

def get_used_gpu_memory():
    command = "nvidia-smi --query-gpu=memory.used --format=csv"
    memory_used_info = sp.check_output(command.split()).decode('ascii').split('\n')[:-1][1:]
    memory_used_values = [int(x.split()[0]) for i, x in enumerate(memory_used_info)]
    return memory_used_values

In [14]:
%time
import numpy as np
import os

print(f"GPU total memory      is {round(get_total_gpu_memory()[0]/1024,2)} GB")
print(f"GPU total used memory is {round(get_used_gpu_memory()[0]/1024,2)} GB")
base_path = os.getcwd()
# Remove the trailing / if need be.

other_changes()

if base_path[-1] in ['/', '\\']:
    base_path = base_path[: - 1]
print("base_path - ",base_path)
print("-"*117)
mode = "train"
print("mode - ",mode)
print("-"*117)
train_json_data , val_json_data = creating_dataset(base_path,mode)

with open(os.path.join(base_path,"data","megadepth","dataset.json"), "w") as file:
    json.dump(train_json_data, file,cls=NumpyEncoder)

# mode = "val"
# print("mode - ",mode)
# print("-"*117)
# json_data = creating_dataset(base_path,mode)

with open(os.path.join(base_path,"data","imw2020val","dataset.json"), "w") as file:
    json.dump(val_json_data, file,cls=NumpyEncoder)

print(f"GPU total used memory is {round(get_used_gpu_memory()[0]/1024,2)} GB")

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs
GPU total memory      is 48.0 GB
GPU total used memory is 0.72 GB
base_path -  /home/hemant/Downloads/amit/ALIKE_code
---------------------------------------------------------------------------------------------------------------------
mode -  train
---------------------------------------------------------------------------------------------------------------------
base_depth_path -  /home/hemant/Downloads/amit/ALIKE_code/data/train
---------------------------------------------------------------------------------------------------------------------
scene_id -  multi-temporal-temple-baalshamin
GPU total used memory is 0.72 GB
---------------------------------------------------------------------------------------------------------------------
scene_id -  dioscuri
GPU total used memory is 1.06 GB
---------------------------------------------------------------------------------------------------------------------
scene_id 

In [15]:
# mode = "val"
# print("mode - ",mode)
# print("-"*117)
# json_data = creating_dataset(base_path,mode)

# with open(os.path.join(base_path,"data","imw2020val","dataset.json"), "w") as file:
#     json.dump(json_data, file,cls=NumpyEncoder)

# print(f"GPU total used memory is {round(get_used_gpu_memory()[0]/1024,2)} GB")

In [25]:
print(get_total_gpu_memory()[0]/1024)
print(get_used_gpu_memory()[0]/1024)
import torch
import gc
gc.collect()
torch.cuda.empty_cache()
print(get_used_gpu_memory()[0]/1024)
print("-"*117)

48.0
38.88671875
38.88671875
---------------------------------------------------------------------------------------------------------------------


In [19]:
os.path.exists("/home/hemant/Downloads/amit/ALIKE_code/data/train/multi-temporal-temple-baalshamin/images/070313_img_6187_syria_palmyra_roman_ruins_999x.png")

True

In [ ]:
070313_img_6187_syria_palmyra_roman_ruins_999x
070313_img_6187_syria-palmyra-roman-ruins_999x.png

In [38]:
print(intrinsics[0])
print(poses[0])
cal_path = "/home/hemant/Downloads/amit/ALIKE_code/data/train/church/dense0/depths/test.h5"
with h5py.File(cal_path, 'r') as f:
    print("f - ",f.keys())
    intrinsic = f['K'][()]
    print("intrinsic - ",intrinsic)
    pose[:3, :3] = f['R'][()]
    pose[:3, 3] = f['T'][()]

NameError: name 'intrinsics' is not defined

In [62]:
save_path = "/home/hemant/Downloads/amit/ALIKE_code/data/train/church/dense0/depths/test.h5"
# img_path = os.path.join(images_path, image)
# print('image size: %d bytes'%os.path.getsize(img_path))
hf = h5py.File(save_path, 'a') # open a hdf5 file
# img_np = np.array(Image.open(img_path))
data={}
data["K"]=intrinsics[0]
data["R"]=poses[0][:3, :3]
data["T"]=poses[0][:3, 3]

dset = hf.create_dataset('K', data=data["K"])  # write the data to hdf5 file
dset = hf.create_dataset('R', data=data["R"])  # write the data to hdf5 file
dset = hf.create_dataset('T', data=data["T"])  # write the data to hdf5 file
hf.close()

In [123]:
d_path = "/home/hemant/Downloads/amit/ALIKE_code/3064466540_f66ed4d28d_o.jpg.h5"
hdf5_file_read = h5py.File(d_path,'r')
print(hdf5_file_read.keys())
gt_depth = hdf5_file_read.get('/SfM_features')
gt_depth = np.array(gt_depth)
print(gt_depth.shape)
hdf5_file_read.close()

<KeysViewHDF5 ['SfM_features']>
(5, 820)


In [77]:
d = list(range(100))
print(d.remove(2))

None


In [137]:
import cv2
imgL = cv2.imread('/home/hemant/Downloads/amit/ALIKE_code/data/train/church/images/00001.jpg',0)
imgR = cv2.imread('/home/hemant/Downloads/amit/ALIKE_code/data/train/church/images/00001.jpg',0)

stereo = cv2.StereoBM_create(numDisparities=96, blockSize=15)
disparity = stereo.compute(imgL,imgR)
disparity.shape

(768, 1024)

In [138]:
imgL[imgL != imgR].shape

(0,)

In [136]:
1024*768

786432